In [ ]:
import re
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
import requests
from pymorphy3 import MorphAnalyzer
from razdel import tokenize
from spacy.lang.ru.stop_words import STOP_WORDS


In [54]:
def get_vacancy(vac_id: str) -> tuple[str, str]:
    r = requests.get(f'https://api.hh.ru/vacancies/{vac_id}', timeout=30)
    vacancy: dict = r.json()
    name = vacancy['name']
    description = vacancy['description']
    return name, description

In [ ]:
def get_vacancies(page: int):
    params = {
        'text': 'Python OR "Программист"',
        'area': 1249, # Омск
        'page': page,
        'per_page': 100
    }
    r1 = requests.get('https://api.hh.ru/vacancies', params)
    vacansies_row: list[dict] = r1.json()['items']
    vacansies_id = [vac['id'] for vac in vacansies_row]
    with ThreadPoolExecutor() as executor:
        result = executor.map(get_vacancy, vacansies_id)
    return result

In [56]:
vacansies = []
vacansies.extend(get_vacancies(page=0))

In [ ]:
vacansies.extend(get_vacancies(page=1))

In [63]:
len(vacansies)

220

In [ ]:
df = pd.DataFrame(columns=['profession', 'description'])
for vacancy in vacansies:
    df.loc[len(df)] = vacancy
df.head(10)

,profession,description
0,UI/UX дизайнер (junior/junior+),"<p><strong>Привет, мы PURRWEB =^._.^=</strong>..."
1,Веб-программист,<p><strong>Сверхуспешная сеть автосервисов REA...
2,Администратор сайта/Контент-менеджер цветочног...,<p>З\п по результатам собеседования</p> <p><st...
3,Android Developer,<p>Создаем собственные продукты и сотрудничаем...
4,Тестировщик,<p>О компании:</p> <p>Мы — ООО &quot;Технологи...
5,Frontend-разработчик,<p>Добрый день. Мы — студия разработки сайтов ...
6,Программист Стажер Битрикс 24,<p>Время прочтения вакансии: 2 минуты</p> <p>Д...
7,Flutter Developer,<p><strong>Если ты предпочитаешь активную разр...
8,Стажер/Intern Системный аналитик/System analys...,<strong>Обязанности:</strong> <ul> <li> <p><st...
9,Тестировщик,"<p>Привет, наш потенциальный крутой тестировщи..."


In [ ]:
# df.to_csv('../../datasets/pz2-hh_parsing/hh_corpus.csv', index=False)

## Предобработка

In [ ]:
morph = MorphAnalyzer(lang='ru')

def preprocess(doc: str) -> list[str]:
    doc = doc.lower()

    doc = re.sub(r'<.*?>', '', doc)

    doc = re.sub(r'[^a-zA-Zа-яА-Я\s]', '', doc)

    doc = [word.text for word in tokenize(doc)]

    doc = [morph.normal_forms(word)[0] for word in doc]

    doc = [word for word in doc if word not in STOP_WORDS]
    return doc

In [27]:
df.description = df.description.apply(preprocess)
df.head(20)

,profession,description
0,UI/UX дизайнер (junior/junior+),"[привет, purrweb, команда, разрабатывать, веб,..."
1,Веб-программист,"[сверхуспешный, сеть, автосервис, reaktor, год..."
2,Администратор сайта/Контент-менеджер цветочног...,"[зп, результат, собеседование, работа, офис, у..."
3,Android Developer,"[создавать, собственный, продукт, сотрудничать..."
4,Тестировщик,"[компания, ооо, quot, технология, сервис, quot..."
5,Frontend-разработчик,"[добрый, день, студия, разработка, сайт, вебди..."
6,Программист Стажер Битрикс 24,"[время, прочтение, вакансия, минута, дать, вак..."
7,Flutter Developer,"[предпочитать, активный, разработка, рутинный,..."
8,Стажер/Intern Системный аналитик/System analys...,"[обязанность, успешно, завести, проект, стажёр..."
9,Тестировщик,"[привет, потенциальный, крутой, тестировщикм, ..."


In [31]:
df.description = df.description.apply(" ".join)
df.to_csv('../../datasets/pz2-hh_parsing/preprocessed_hh_corpus.csv', index=False)